1. selenium으로 토스 GoolePlay에 접속
2. 전체리뷰 보기 클릭
3. 리뷰창이 뜨면 최신순으로 정렬
4. 마우스 스크롤을 해서 오늘부터 2년전 날짜가 나올 때까지 스크롤
5. 데이터 수집
6. csv파일로 저장 → DB 저장
7. 수집코드 함수화
8. KB, 신한, 하나, 뱅크샐러드, 카카오뱅크 리뷰도 저장

In [1]:
import time
import pandas as pd
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

from general_dbio import *
print(dir())

['In', 'Out', '_', '__', '___', '__builtin__', '__builtins__', '__doc__', '__loader__', '__name__', '__package__', '__spec__', '_dh', '_i', '_i1', '_ih', '_ii', '_iii', '_oh', 'create_engine', 'datetime', 'exit', 'get_ipython', 'open', 'pd', 'pymysql', 'quit', 'relativedelta', 'saveDB', 'time', 'timedelta']


In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

options = Options()
options.add_experimental_option("detach", True)
options.add_argument('start-maximized')

# 사람인척하기위해서 개발자도구 network에서 user-agent추가, 언어 추가
options.add_argument('Chrome/134.0.0.0')
options.add_argument('lang=ko_KR')

# gpu 안쓰기
options.add_argument('--disable-gpu') 

In [3]:
end_date = datetime.today() - relativedelta(years=2) - timedelta(days=1)
end_date_text = end_date.strftime('%Y년 %m월 %d일')
print(end_date)
print(end_date_text)
print( datetime.today() >= datetime.strptime(end_date_text, '%Y년 %m월 %d일'))

2023-04-13 23:08:29.909200
2023년 04월 13일
True


In [23]:
# shinhan_sol='com.shinhan.sbanking'
# toss='viva.republica.toss'
bank_list = dict(kb_star='com.kbstar.kbbank', hana='com.kebhana.hanapush', banksalad='com.rainist.banksalad2', kakaobank='com.kakaobank.channel')
bank_list

{'kb_star': 'com.kbstar.kbbank',
 'hana': 'com.kebhana.hanapush',
 'banksalad': 'com.rainist.banksalad2',
 'kakaobank': 'com.kakaobank.channel'}

In [20]:
from selenium.common.exceptions import NoSuchElementException

def safe_find_text(idx, bank_name, element_name, selector, by=By.CSS_SELECTOR, driver=None):
    try:
#         print(f'idx 입니다 : {idx}')
#         print(f'bank name 입니다 : {bank_name}')
#         print(f'element name 입니다 : {element_name}')
#         print(f'selector 출력해보는중!!! {selector}')
#         print(f'return값 출력해보는 중!!! {driver.find_element(by, selector).text}')
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((by, selector))
        )
        return element.text
    except NoSuchElementException:
        print(f'{bank_name}에서 {idx} : {element_name} 수집 중')
        return None
    except Exception as e:
        print(f'{bank_name}에서 {element_name} 수집 중')
        print(f"⚠️ 예외 발생: {e} | selector: {selector}")
        return None

def safe_find_attr(idx, bank_name, element_name, selector, attr, by=By.CSS_SELECTOR, driver=None):
    try:
        return float(driver.find_element(by, selector).get_attribute(attr)[-10])
    except NoSuchElementException:
        return None
    except Exception as e:
        print(f'{bank_name}에서 {element_name} 수집 중')
        print(f"⚠️ 예외 발생: {e} | selector: {selector} | attr: {attr}")
        return None


In [14]:
driver = webdriver.Chrome(
service=Service(ChromeDriverManager().install()),
options=options
)

driver.get(f'https://play.google.com/store/apps/details?id=viva.republica.toss')

reviews_btn = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "button[aria-label*='평점 및 리뷰 자세히 알아보기']"))
        )
reviews_btn.click()
order_btn = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, "#sortBy_1"))
    )
order_btn.click()
recent_order = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, 'span[aria-label*="최신"]'))
    )
recent_order.click()

reviews_modal = driver.find_element(By.CSS_SELECTOR, '#yDmH0d > div.VfPpkd-Sx9Kwc.cC1eCc.UDxLd.PzCPDd.HQdjr.VfPpkd-Sx9Kwc-OWXEXe-FNFY6c > div.VfPpkd-wzTsW > div > div > div > div > div.fysCi.Vk3ZVd')

In [9]:
driver.find_element(By.CSS_SELECTOR, f'div.RHo1pe:nth-of-type(1) header > div div.X5PpBb').text

'유7080'

In [24]:
columns = ['날짜', '닉네임','평점','리뷰내용']

for bank, domain in bank_list.items():
    idx = 1
    driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options
    )

    driver.get(f'https://play.google.com/store/apps/details?id={domain}')

    reviews_btn = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "button[aria-label*='평점 및 리뷰 자세히 알아보기']"))
        )
    reviews_btn.click()
    order_btn = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "#sortBy_1"))
        )
    order_btn.click()
    recent_order = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, 'span[aria-label*="최신"]'))
        )
    time.sleep(1)
    recent_order.click()


    reviews_modal = driver.find_element(By.CSS_SELECTOR, '#yDmH0d > div.VfPpkd-Sx9Kwc.cC1eCc.UDxLd.PzCPDd.HQdjr.VfPpkd-Sx9Kwc-OWXEXe-FNFY6c > div.VfPpkd-wzTsW > div > div > div > div > div.fysCi.Vk3ZVd')

    try: 
        while True:
            time.sleep(1)
            date = safe_find_text(idx, bank, 'date', f'div.RHo1pe:nth-of-type({idx}) header span.bp9Aid', driver=driver)
            nickname = safe_find_text(idx, bank, 'nickname', f'div.RHo1pe:nth-of-type({idx}) header > div div.X5PpBb', driver=driver)
            rating = safe_find_attr(idx, bank, 'rating', f'div.RHo1pe:nth-of-type({idx}) header .Jx4nYe > div', 'aria-label', driver=driver)
            content = safe_find_text(idx, bank, 'content', f'div.RHo1pe:nth-of-type({idx}) div.h3YV2d', driver=driver)

            print(idx, date, nickname, rating, content, end='\r')
            
            if not date:
                continue

            date_cast = datetime.strptime(date, '%Y년 %m월 %d일')
            if date_cast <= end_date:
                break

            temp = (date, nickname, rating, content)
            df = pd.DataFrame([temp], columns=columns)
            saveDB(df, 'google_play_reviews', bank)
            time.sleep(1)
            try:
                driver.execute_script("arguments[0].scrollTop += 1000;", reviews_modal)  # 아래로 500px씩 스크롤
                time.sleep(1)
            except:
                print('스크롤 안됨')

            idx += 1
    except Exception as e:
        print(f'예외 발생: {type(e).__name__} - {e}')
    finally:
        driver.quit()

kb_star에서 rating 수집 중m Jong Taik 5.0 사용편리함, 안전성 우수감사드립니다.은행에 직접 가지않아도 될정도로 쉬웠습니다번깔고 폰재부팅해도 계속안됨. 어플 왜만든거야 짜증나.업데이트 하라고해도 안뜨자나!!!!재부팅 이런 것만 제시해서 좀 난감했네요. UCWB 무슨 에러였는데.. 사유와 더불어 창구로 가란 메시지로 바꿔주면 좋을 것 같습니다.버렸네요 이러면 국민 오픈뱅킹해지하고 타은행 오픈을 만들어 쓰는게 낫죠다. 이틀이 경과한 2023년 10월 22일 업데이트로 오류가 사라졌지만 어떤 사과와 해명도 없습니다. 우선 왜 휴일을 앞둔 금요일에 이런 업데이트를 진행했는지 설명바랍니다. 주중에 업데이트 했다면 다음 날 조치할 수 있습니다. 무엇보다 고객들도 다음 날 금융거래를 이어갈 수 있구요. 그리고 왜 금융업무가 몰리는 월 하순에 이런 일을 저지른 건가요? 당연히 오류에 따른 피해도 증가할 수 밖에 없습니다. 분명한 사과와 해명이 필요합니다.작합시다..! 기본도 안되어 있는데 무슨 다른 기능이 필요합니까.!!가? 수상한 은행이네
⚠️ 예외 발생: HTTPConnectionPool(host='localhost', port=4120): Read timed out. (read timeout=120) | selector: div.RHo1pe:nth-of-type(10076) header .Jx4nYe > div | attr: aria-label
스크롤 안됨2023년 9월 6일 ᄋ우왕 5.0 알려주신대로 생체인증등록을 하니 위젯이 진짜 좋긴 하네요. 위젯에 처음뜨는 카드는 최근 사용한 카드라는 점도 알았습니다. 바라는 점이 있다면 위젯으로 결제시 실제 최근 결제한 카드가 아닌 최근에 지문인식을 한 카드로 결제되게 바꿔주신다면 더욱 편리해질 것 같습니다. 결제카드를 바꾸기 위해서는 결제 한번이 이루워져야 한다는 단점을 개선 할 수 있으니까요. 답변해주셔서 감사합니다.만드는 프로세스 중에 계좌내역을 볼 수 없어서 그 부분에 불편함을 말한 것입니다.일OT

SessionNotCreatedException: Message: session not created: probably user data directory is already in use, please specify a unique value for --user-data-dir argument, or don't use --user-data-dir
Stacktrace:
	GetHandleVerifier [0x009C8073+60707]
	GetHandleVerifier [0x009C80B4+60772]
	(No symbol) [0x007F0683]
	(No symbol) [0x0082369F]
	(No symbol) [0x0081F4A9]
	(No symbol) [0x00868D63]
	(No symbol) [0x0086877A]
	(No symbol) [0x0085CE46]
	(No symbol) [0x0082C5D3]
	(No symbol) [0x0082D424]
	GetHandleVerifier [0x00C0BB53+2435075]
	GetHandleVerifier [0x00C070F3+2416035]
	GetHandleVerifier [0x00C2349C+2531660]
	GetHandleVerifier [0x009DF145+155125]
	GetHandleVerifier [0x009E5AED+182173]
	GetHandleVerifier [0x009CF948+91640]
	GetHandleVerifier [0x009CFAF0+92064]
	GetHandleVerifier [0x009BA5B0+4704]
	BaseThreadInitThunk [0x7712FCC9+25]
	RtlGetAppContainerNamedObjectPath [0x774C82AE+286]
	RtlGetAppContainerNamedObjectPath [0x774C827E+238]


In [12]:
driver.quit()